In [ ]:
import numpy as np
from numpy.random import RandomState

from fgn import fbm

import os, re

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [ ]:
N = 2**15 + 1
generator1 = fbm( N = N, H = .80 )
generator1.set_rnd( RandomState( ) )
T1, X1 = generator1( )

generator2 = fbm( N = N, H = .95 )
generator2.set_rnd( RandomState( ) )
T2, X2 = generator2( )

In [ ]:
## Paste the sample paths
X = np.concatenate( ( X1[:-1], X2 + X1[ -1 ] ) )
T = np.concatenate( ( T1[:-1], T2 + T1[ -1 ] ) )

In [ ]:
print np.subtract( *np.percentile( np.diff( X ), [ 75, 25 ] ) )
print np.std( np.diff( X ) )
print np.median( np.abs( np.diff( X ) ) )

In [ ]:
# T = np.sort( np.random.uniform( size = 2**10 + 1 ) )

In [ ]:
from crossing_tree import xtree_build, xtree_build_old
window = 0.005
L = int( T[ -1 ] / ( 2 * window ) )
## Prepare the sample times for running hurst estimation. Allow at
##  most "window" overlap between data used for the estimates.
## Do not make assumptions about the spacing of the times in (T, X).
##  Except that it is sorted in ascending order.
bins = np.concatenate( ( np.arange( T[ 0 ], T[ -1 ], window ), [ np.inf ] ) )
binx = np.searchsorted( T, bins, side = 'left' )
hurst = np.empty( len( binx ) - L, dtype = np.float )
waiting = hurst.copy( )
for i,j,k in zip( binx[:-L], binx[L:], range( hurst.shape[ 0 ] ) ) :
    S, Y = T[ i : j ], X[ i : j ]
    delta = ( 2**5 ) * np.median( np.abs( np.diff( Y ) ) )
    Tnk, Xnk, Znk, Vnk, Wnk = xtree_build( S, Y, delta = delta, max_height = 3 )
    hurst[ k ] = np.log( 2 ) / np.log( np.average( np.concatenate( Znk[1:] ) ) )
    waiting[k] = np.average( np.concatenate( Wnk[1:] ) )

In [ ]:
print np.log( np.average( Wnk[1] ) ) + np.log( np.average( np.concatenate( Znk[1:] ) ) )

A process $X(t)$ is self-similar with hurst exponent $H$ is 
$$ X(a t) \overset{\mathcal{D}}{=} a^H X(t)$$ for all $t\in [0,1]$ and $a>0$, where equality is understood as equality of all finite vector distributions.
Since $a$ is arbitrary positive real number the self-similarity definition is equivalent to
$$ X(\frac{t}{a}) \overset{\mathcal{D}}{=} a^{-H} X(t)$$
which reduces to
$$ X(s) \overset{\mathcal{D}}{=} a^{-H} X(a s)$$

For the crossing tree
$$ X(s) \overset{\mathcal{D}}{=} 2^{-k} X( \mu^k s) = (\mu^k)^{- \frac{\log 2}{\log \mu}} X( \mu^k s)$$
whence the following Hurst exponent estimator could be constructed:
$$\hat{H} = \frac{\log 2}{\log \mu} $$

In [ ]:
Znk

In [ ]:
plt.figure( figsize = (16,6) )
ax = plt.subplot(121)
ax.plot( hurst )


ax = plt.subplot(122)
ax.plot( np.log( waiting ) )

# plt.plot( waiting )
# plt.

In [ ]:
plt.plot( T, X )

In [ ]:
# def running_hurst( T, X, window = .05 ) :
index = np.nonzero( ( T > window ) & ( T < 1 - window ) )[ 0 ]
for i in index :
## Select the 
    mask = ( np.abs( T - T[ i ] ) <= window )
    delta = ( 2**3 ) * np.median( np.abs( np.diff( X[ mask ] ) ) )

In [ ]:
Tnk, Xnk, Znk, Vnk, Wnk = xtree_build( T[ mask ], X[ mask ], delta = delta, max_height = 6 - 3 )

In [ ]:
np.log( 2 ) / np.log( np.average( np.concatenate( Znk[1:] ) ) )

In [ ]:
Cjn = np.reshape( np.sum( Djnk, axis = ( 2, ), dtype = np.float ), Djnk.shape[ :-1 ] + ( 1, ) )
Zk = 2 * np.reshape( 1 + np.arange( Djnk.shape[ -1 ] ), ( 1, 1, Djnk.shape[ -1 ] ) )
Pjnk = np.where( Cjn > 0, Djnk / Cjn, 0 )
Mjn = np.reshape( np.sum( Pjnk * Zk, axis = ( 2, ) ), Cjn.shape )
Hjn = np.log( 2 ) / np.log( Mjn )

In [ ]:
delta = np.subtract( *np.percentile( np.diff( X ), [ 75, 25 ] ) )
# delta = np.std( np.diff( X ) )
# delta = np.median( np.abs( np.diff( X ) ) )
Tnk, Xnk, Znk, Vnk, Wnk = xtree_build( T, X, delta = delta )